#Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers -q
!pip install nltk
!pip install pandarallel 

     |████████████████████████████████| 3.4 MB 12.6 MB/s 
     |████████████████████████████████| 895 kB 38.2 MB/s 
     |████████████████████████████████| 3.3 MB 44.3 MB/s 
     |████████████████████████████████| 596 kB 45.5 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
  Created wheel for pandarallel: filename=pandarallel-1.5.4-py3-none-any.whl size=18690 sha256=6530d89d30a7bef2c79f81ef7417f777623a39acf7e4cbb9269120cf481ddb19
  Stored in directory: /root/.cache/pip/wheels/71/de/ad/759faf4fea1e9b0fd935da28847aa3ca2eef14a888c2dc4952
Successfully built pandarallel


In [3]:
! pip install annoy -q

     |████████████████████████████████| 646 kB 12.3 MB/s 


In [4]:
# Importing Libraries
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
import string
import re
from tqdm.auto import tqdm
tqdm.pandas()

import _pickle as cPickle
import nltk
nltk.download('punkt')

# Checking to see if CUDA is available if so storing details in device

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import psutil

workers = psutil.cpu_count()
# psutil.cpu_count(logical=False)

In [6]:
import pandas as pd

from datetime import datetime
from collections import Counter
import torch
from transformers import AutoTokenizer, AutoModel, BertModel
from sklearn.metrics.pairwise import cosine_similarity


import re  
from gensim.models import Word2Vec    #Import gensim Word2Fec
from sklearn.decomposition import PCA #Grab PCA functions
import numpy as np

#Plot helpers
import matplotlib.pyplot as plt

#Enable matplotlib to be interactive (zoom etc)
import ast

from pandarallel import pandarallel

from pandarallel.utils import progress_bars
progress_bars.is_notebook_lab = lambda : True

pandarallel.initialize(progress_bar=True, nb_workers=workers, use_memory_fs=False)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


#Loading Data and Lexicon

In [7]:
#new ones_collected by passing through the emobert
# df = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/NRC Emotion Intensity.csv")
# df = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/NRC-Emotion-Intensity-Lexicon-v2.csv")
# df = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/NRC-Emotion-Intensity-Lexicon-lda.csv")
df = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/NRC-Emotion-Intensity-Lexicon-without_trust.csv")

# df['embedding'] = df['embedding'].tolist()
df['embedding'] = [ast.literal_eval(i) for i in df['embedding'].values.tolist()]
print(df.head())
print(df.columns)
df = df.dropna()

         word  ...                                          embedding
0    aaaaaaah  ...  [0.3540674149990082, 0.7942342162132263, 0.423...
1   abandoned  ...  [-0.5577418804168701, 0.015890369191765785, -0...
2  aberration  ...  [-0.5287905931472778, -0.1950991153717041, -0....
3       abhor  ...  [-0.6878913640975952, 0.1946220099925995, -1.3...
4   abhorrent  ...  [-0.3238641023635864, 0.3315010368824005, -1.2...

[5 rows x 4 columns]
Index(['word', 'emotion', 'emotion-intensity-score', 'embedding'], dtype='object')


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2273 entries, 0 to 2272
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   word                     2273 non-null   object 
 1   emotion                  2273 non-null   object 
 2   emotion-intensity-score  2273 non-null   float64
 3   embedding                2273 non-null   object 
dtypes: float64(1), object(3)
memory usage: 88.8+ KB


In [9]:
len(df['embedding'][0])

768

In [10]:
from annoy import AnnoyIndex
import random

f = 768
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed

# super fast, will just mmap the file
# t.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/angular_nrc_embedding_space_V6.ann') 
# t.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/angular_nrc_embedding_space_words_lda.ann') 
t.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/angular_nrc_embedding_space_words_without_trust.ann') 

True

In [11]:
t.get_n_items()

2273

#WelFake

In [17]:
dff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/WELFake/WELFake.csv")
dff = dff[:25000]

In [18]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      25000 non-null  int64 
 1   title   24845 non-null  object
 2   text    24988 non-null  object
 3   label   25000 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 781.4+ KB


In [19]:
dff['total_text'] = dff['title']+ ' ' + dff['text']
dff['total_text'][0]

'LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO] No comment is expected from Barack Obama Members of the #FYF911 or #FukYoFlag and #BlackLivesMatter movements called for the lynching and hanging of white people and cops. They encouraged others on a radio show Tuesday night to  turn the tide  and kill white people and cops to send a message about the killing of black people in America.One of the F***YoFlag organizers is called  Sunshine.  She has a radio blog show hosted from Texas called,  Sunshine s F***ing Opinion Radio Show. A snapshot of her #FYF911 @LOLatWhiteFear Twitter page at 9:53 p.m. shows that she was urging supporters to  Call now!! #fyf911 tonight we continue to dismantle the illusion of white Below is a SNAPSHOT Twitter Radio Call Invite   #FYF911The radio show aired at 10:00 p.m. eastern standard time.During the show, callers clearly call for  lynching  and  killing  of white people.A 2:3

In [20]:
dff.dropna(subset = ['total_text'], how='all', inplace=True)
dff = dff[['label','total_text','id']]
dff.reset_index(drop=True,inplace=True)

In [21]:
dff.head()

,label,total_text,id
0,1,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,0
1,1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,2
2,0,"Bobby Jindal, raised Hindu, uses story of Chri...",3
3,1,SATAN 2: Russia unvelis an image of its terrif...,4
4,1,About Time! Christian Group Sues Amazon and SP...,5


In [22]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24833 entries, 0 to 24832
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       24833 non-null  int64 
 1   total_text  24833 non-null  object
 2   id          24833 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 582.1+ KB


#ISOT

In [ ]:
dff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/ISOT/ISOT.csv")
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
 5   id       44898 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 2.1+ MB


In [ ]:
dff.head()

,title,text,subject,date,label,id
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0,0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0,2
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0,3
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0,4


In [ ]:
dff['total_text'] = dff['title']+ ' ' + dff['text']

In [ ]:
dff['total_text'][30]

'Alabama to certify Democrat Jones winner of Senate election (Reuters) - Democrat Doug Jones’ surprise victory over Republican Roy Moore in this month’s special U.S. Senate election will be certified on Dec. 28, Alabama state officials said on Friday. Jones will be the first Democrat sent to the Senate from Republican stronghold Alabama in a quarter century. When he takes office, Republicans’ majority in the chamber will narrow to 51 of the 100 seats. Alabama Governor Kay Ivey, Attorney General Steve Marshall and Secretary of State John Merrill will meet to certify Jones’ win, Merrill’s office said in a statement. Jones’ margin of victory was 1.5 percentage points. Moore has not conceded defeat in the Dec. 12 vote, despite being urged by President Donald Trump to do so. Calls and emails to Moore’s campaign spokeswomen were not immediately returned on Friday. Moore was a controversial candidate whose campaign was beset by allegations that he sexually assaulted or pursued teenage girls w

In [ ]:
dff = dff[['label','total_text','id']]

In [ ]:
dff.dropna(subset = ['total_text'], how='all', inplace=True)
dff.reset_index(drop=True,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#Kaggle

In [ ]:
dff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/Kaggle/Kaggle.csv")
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  object
 5   id       44898 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 2.1+ MB


In [ ]:
dff.head()

,title,text,subject,date,label,id
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Fake,0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Fake,1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Fake,2
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Fake,3
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Fake,4


In [ ]:
dff['total_text'] = dff['title']+ ' ' + dff['text']
dff['total_text'][30]

' BREAKING: Cop Finally Gets His Due, Walter Scott’s Killer Sentenced To Prison (DETAILS) In America, we have been having a conversation about police brutality against black Americans. Despite the countless black people murdered unjustly by police, there is usually no justice. Sandra Bland, Philando Castile, Keith Lamont Scott, Michael Brown, Freddie Gray too many to mention here, really. All of those people were senselessly murdered by cops who chose to be their judges, juries, and executioners, and they did so with impunity and without consequence. However, there is hope, and it is coming out of South Carolina, of all places.North Charleston police officer Michael Slager murdered Walter Scott, a black man who was fleeing after a routine traffic stop in cold blood in 2015. He would have gotten away with it, too, had it not been for a citizen who was brave enough to tape the murder. The tape showed that Slager had lied about his life being in danger, and it showed him cuffing Scott s l

In [ ]:
dff = dff[['label','total_text','id']]

In [ ]:
dff.dropna(subset = ['total_text'], how='all', inplace=True)
dff.reset_index(drop=True,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#Kaggle Real_Fake

In [ ]:
dff = pd.read_csv("/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/Kaggle_real_fake/fake_or_real_news.csv")
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   6335 non-null   object
 1   text    6335 non-null   object
 2   label   6335 non-null   object
 3   id      6335 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [ ]:
dff.head()

,title,text,label,id
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,0
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,1
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,2
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,3
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,4


In [ ]:
dff['total_text'] = dff['title']+ ' ' + dff['text']
dff['total_text'][0]

'You Can Smell Hillary’s Fear Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. \nThe FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. \nThe FBI’s leadership i

In [ ]:
dff = dff[['label','total_text','id']]

In [ ]:
dff.dropna(subset = ['total_text'], how='all', inplace=True)
dff.reset_index(drop=True,inplace=True)

#Data Cleaning

In [23]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24833 entries, 0 to 24832
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       24833 non-null  int64 
 1   total_text  24833 non-null  object
 2   id          24833 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 582.1+ KB


In [24]:
def lower(text):
  return text.lower()

dff['total_text'] = dff['total_text'].apply(lower)

In [25]:
# Remove Duplicates

duplicate_count = len(dff['total_text'])-len(dff['total_text'].drop_duplicates())

print('Duplicate count:', duplicate_count)
print('Total records before remove duplicates:', dff.shape[0])


dff = dff.drop_duplicates(subset='total_text', keep="first")

print('updated record count:', dff.shape[0])

Duplicate count: 1043
Total records before remove duplicates: 24833
updated record count: 23790


In [26]:
tokenizer = nltk.data.load('tokenizers/punkt/PY3/english.pickle')
  
def split_token(text):
  return tokenizer.tokenize(text)

dff['tokens'] = dff['total_text'].apply(split_token)

In [27]:
import string
import re
import nltk.data

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_nonascii(sent):
  return "".join([i for i in sent if i.isascii()])

def remove_punctuations(text):
  res = re.sub(r'[^\w\s]', ' ', text)
  return res

def remove_emoji(string):
    '''
    Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
    remove emojis
    '''
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def strip_all_tags(text):
    '''
    uses this to remove any mentions and hash tags
    '''
    entity_prefixes = ['@','#']
    words = []
    for word in text.split():
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

def remove_num(text):
  return "".join([c for c in text if not c.isdigit()])

def remove_mul_space(text):
  return " ".join(text.split())

In [28]:
def clean(tokens):
  refined_tokens = []
  for i in range(len(tokens)):
    tokens[i] = remove_urls(tokens[i])
    tokens[i] = remove_emoji(tokens[i])
    tokens[i] = strip_all_tags(tokens[i])
    tokens[i] = remove_nonascii(tokens[i])
    tokens[i] = remove_punctuations(tokens[i])
    tokens[i] = remove_num(tokens[i])
    tokens[i] = remove_mul_space(tokens[i])
    refined_tokens.append(tokens[i])
  return refined_tokens


dff['refined_tokens'] = dff['tokens'].apply(clean)
dff = dff.reset_index(drop=True)
dff['index'] = dff.index

In [29]:
dff.head()

,label,total_text,id,tokens,refined_tokens,index
0,1,law enforcement on high alert following threat...,0,[law enforcement on high alert following threa...,[law enforcement on high alert following threa...,0
1,1,unbelievable! obama’s attorney general says mo...,2,"[unbelievable, obamas attorney general says mo...","[unbelievable, obamas attorney general says mo...",1
2,0,"bobby jindal, raised hindu, uses story of chri...",3,[bobby jindal raised hindu uses story of chris...,[bobby jindal raised hindu uses story of chris...,2
3,1,satan 2: russia unvelis an image of its terrif...,4,[satan russia unvelis an image of its terrifyi...,[satan russia unvelis an image of its terrifyi...,3
4,1,about time! christian group sues amazon and sp...,5,"[about time, christian group sues amazon and s...","[about time, christian group sues amazon and s...",4


In [30]:
dff['refined_tokens'][1]

['unbelievable',
 'obamas attorney general says most charlotte rioters were peaceful protestersin her home state of north carolina video now most of the demonstrators gathered last night were exercising their constitutional and protected right to peaceful protest in order to raise issues and create change',
 'loretta lynch aka eric holder in a skirt']

In [ ]:
# # Next we remove stop words and short words.
# # For stop words we use the SMART stop word list here
# # Initially we import the text file and create the list.

# # stop = []
# # with open("/content/drive/Shareddrives/FYP - knk/Resources/SMART_STOP_WORDS.txt", "r") as f:
# #   for word in f:
# #       # Here we remove the apostrophe as well
# #       stop.append(word.strip().replace("'",""))
# # stop.extend(['nan', 'coronavirus','covid','covid19', 'vaccine', 'false', 'selfcovidvaccine', 'vaccines', 'vaccinevaccine', 'corona', 'pandemic'])

# dff['refined_tokens'] = dff['tokens'].apply(remove_stop_words_and_short_words)


In [31]:
total_tokens = 0
total_length = len(dff)
for index, row in dff.iterrows():
  total_tokens+=len(row['refined_tokens'])

average_length = total_tokens/total_length
print(average_length)

22.577553593947037


In [32]:
len(dff['refined_tokens'][5])

1

#Mean Pooling

In [33]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    # print('ime',input_mask_expanded)
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    # print('se',sum_embeddings)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [34]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [35]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/go_model")
model = AutoModel.from_pretrained("/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/go_model")

Some weights of the model checkpoint at /content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Resources/go_model were not used when initializing DistilBertModel: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
model.to(device)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [37]:
def get_mean_pooling_emb_tokenized(sentences):
  encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')
  encoded_input.to(device)
  #Compute token embeddings
  with torch.no_grad():
      model_output = model(**encoded_input)

  sentence_embeddings_raw = mean_pooling(model_output, encoded_input['attention_mask'])
  sentence_embeddings = sentence_embeddings_raw.tolist()

  return sentence_embeddings 

In [38]:
def get_mean_pooling_emb(sentences):
  encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')
  encoded_input.to(device)
  #Compute token embeddings
  with torch.no_grad():
      model_output = model(**encoded_input)

  sentence_embeddings_raw = mean_pooling(model_output, encoded_input['attention_mask'])
  sentence_embeddings = sentence_embeddings_raw.tolist()

  return sentence_embeddings[0]

In [39]:
len(get_mean_pooling_emb_tokenized(dff['refined_tokens'][0]))

47

In [ ]:
# Generate Embeddings
dff['embedding_tokenized']  = dff['refined_tokens'].progress_apply(get_mean_pooling_emb_tokenized)

  0%|          | 0/23790 [00:00<?, ?it/s]

In [ ]:
dff['embedding_total_text']  = dff['total_text'].progress_apply(get_mean_pooling_emb)

In [ ]:
len(dff['embedding_tokenized'][0])

In [ ]:
dff.tail()

In [ ]:
dff.info()

#Emotion Extraction Pipeline

In [ ]:
pandarallel.initialize()

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
def get_nearest_neighbours_tokenized(index):
    # t1 = datetime.now()

    total_dict = []
    for i in range(len(dff['refined_tokens'][index])):
      tuples = []
      
      embeding = dff['embedding_tokenized'][index][i]
      sentence_tokens = dff['refined_tokens'][index][i].split(' ')

      # for i,row_e in df.iterrows():
          
      #     dis = cosine_similarity([row_e['embedding']], [embeding])
      #     # print([row_e['tokens'],row_d['tokens'],dis])
      #     tuples.append([row_e['word'],row_e['emotion'],dis,row_e['embedding'],row_e['emotion-intensity-score']])

      indexes, distances = t.get_nns_by_vector(embeding, 50, include_distances=True)
      for i in range(len(indexes)):
        tuples.append([df['word'][indexes[i]],df['emotion'][indexes[i]],distances[i],df['embedding'][indexes[i]],df['emotion-intensity-score'][indexes[i]]
                      #  df['Positive'][indexes[i]],df['Negative'][indexes[i]]
                      ])
      # print(indexes)
      # print(distances)
      # print(tuples)
      
      s_tup = sorted(tuples, key=lambda x: x[2])#sort tuples based on the cosine distance
      neaarest_neighbs_words = []
      neaarest_neighbs_embs = []
      neaarest_neighbs_labels = []
      neaarest_neighbs_distance = []
      # neaarest_neighbs_positive = []
      # neaarest_neighbs_negative = []
      neaarest_neighbs_intensity_score = []
      for i,m in enumerate(s_tup[::-1]):
          # print(m)
          if(i<50):#getting the nearest 50 neighbours
              neaarest_neighbs_words.append(m[0])
              neaarest_neighbs_embs.append(m[3])
              neaarest_neighbs_labels.append(m[1])
              # distance = m[2].tolist()[0][0]
              distance = m[2]
              neaarest_neighbs_distance.append(distance)
              neaarest_neighbs_intensity_score.append(m[4])
              # neaarest_neighbs_positive.append(m[5])
              # neaarest_neighbs_negative.append(m[6])
    

      n_score_dict = calculate_scores(neaarest_neighbs_words,neaarest_neighbs_labels,neaarest_neighbs_distance,neaarest_neighbs_intensity_score
                                      )
      
      for key in n_score_dict:
        n_score_dict[key] = round((n_score_dict[key]/len(sentence_tokens)),4)
      
      # neighbour_output = [n_score_dict,{'words':neaarest_neighbs_words,'embs':neaarest_neighbs_embs,'labels':neaarest_neighbs_labels}]
      # normalized_score_dict = neighbour_output[0]
      
      n_score_dict = {k: round(v,3) for k, v in sorted(n_score_dict.items(), key=lambda item: item[1])}
      total_dict.append(n_score_dict)

    return total_dict

In [ ]:
import operator

def dict_to_result_tokenized(list_emotion_dict):
  high_8 = []
  for i in range(len(list_emotion_dict)):
    emotion_dict = list_emotion_dict[i]
    if (all(value == 0 for value in emotion_dict.values())):
      highest_8 = None
    else:
      highest_8 = max(emotion_dict.items(), key=operator.itemgetter(1))[0]
    high_8.append(highest_8)
  return high_8

# result = emotion_candidates_recognition('That was good',1)
# x = dict_to_result(result)

In [ ]:
def get_nearest_neighbours(index):
    # t1 = datetime.now()
    tuples = []
    
    embeding = dff['embedding_total_text'][index]
    sentence_tokens = dff['total_text'][index].split(' ')

    # for i,row_e in df.iterrows():
        
    #     dis = cosine_similarity([row_e['embedding']], [embeding])
    #     # print([row_e['tokens'],row_d['tokens'],dis])
    #     tuples.append([row_e['word'],row_e['emotion'],dis,row_e['embedding'],row_e['emotion-intensity-score']])

    indexes, distances = t.get_nns_by_vector(embeding, 50, include_distances=True)
    for i in range(len(indexes)):
      tuples.append([df['word'][indexes[i]],df['emotion'][indexes[i]],distances[i],df['embedding'][indexes[i]],df['emotion-intensity-score'][indexes[i]]
                    #  df['Positive'][indexes[i]],df['Negative'][indexes[i]]
                     ])
    # print(indexes)
    # print(distances)
    # print(tuples)
    
    s_tup = sorted(tuples, key=lambda x: x[2])#sort tuples based on the cosine distance
    neaarest_neighbs_words = []
    neaarest_neighbs_embs = []
    neaarest_neighbs_labels = []
    neaarest_neighbs_distance = []
    # neaarest_neighbs_positive = []
    # neaarest_neighbs_negative = []
    neaarest_neighbs_intensity_score = []
    for i,m in enumerate(s_tup[::-1]):
        # print(m)
        if(i<50):#getting the nearest 50 neighbours
            neaarest_neighbs_words.append(m[0])
            neaarest_neighbs_embs.append(m[3])
            neaarest_neighbs_labels.append(m[1])
            # distance = m[2].tolist()[0][0]
            distance = m[2]
            neaarest_neighbs_distance.append(distance)
            neaarest_neighbs_intensity_score.append(m[4])
            # neaarest_neighbs_positive.append(m[5])
            # neaarest_neighbs_negative.append(m[6])

    n_score_dict = calculate_scores(neaarest_neighbs_words,neaarest_neighbs_labels,neaarest_neighbs_distance,neaarest_neighbs_intensity_score
                                    # neaarest_neighbs_positive,neaarest_neighbs_negative
                                    )
    
    neighbour_output = [n_score_dict,{'words':neaarest_neighbs_words,'embs':neaarest_neighbs_embs,'labels':neaarest_neighbs_labels}]
    normalized_score_dict = neighbour_output[0]
  
    for key in normalized_score_dict:    
      normalized_score_dict[key] = round((normalized_score_dict[key]/len(sentence_tokens)),4)

    normalized_score_dict = {k: round(v,3) for k, v in sorted(normalized_score_dict.items(), key=lambda item: item[1])}
    # print(normalized_score_dict)
    return normalized_score_dict

In [ ]:
def calculate_scores(neaarest_neighbs_words,neaarest_neighbs_labels,neaarest_neighbs_distance,neaarest_neighbs_intensity_score
                    #  ,neaarest_neighbs_positive,neaarest_neighbs_negative
                     ):
  score_dict = {
              'anticipation':0,
              'anger':0,
              'fear':0,
              'sadness':0,
              'trust':0,
              'joy':0,
              'surprise':0,
              'disgust':0
              # 'positive':0,
              # 'negative':0
              }

  #Scoring Mechanism
  for i in range(0,len(neaarest_neighbs_words)):
    distance = 0
    if neaarest_neighbs_distance[i] == 0 :
      distance = 1
    else :
      distance = (1/neaarest_neighbs_distance[i])
    
    score = (distance*(neaarest_neighbs_intensity_score[i]))  

    score_dict[neaarest_neighbs_labels[i]]=score_dict[neaarest_neighbs_labels[i]]+score

  # score_dict['positive']=score_dict['anger']+ score_dict['fear']+score_dict['disgust']+score_dict['sadness']
  # score_dict['negative']=score_dict['joy']+score_dict['trust']+score_dict['anticipation']+score_dict['surprise']




  #Normalising Mechanism
  normalized_score_dict = score_dict.copy()
  # for k in score_dict.keys():
  #   if score_dict[k] ==0:
  #     continue
  #     # del normalized_score_dict[k]
  #   else:
  #     normalized_score_dict[k]
  #     # normalized_score_dict[k] = round((score_dict[k]/score_max),3)

  return normalized_score_dict

In [ ]:
import operator

def dict_to_result(emotion_dict):
  # del emotion_dict['positive']
  # del emotion_dict['negative']
  if (all(value == 0 for value in emotion_dict.values())):
    highest_8 = None
  else:
    highest_8 = max(emotion_dict.items(), key=operator.itemgetter(1))[0]
  return highest_8

# result = emotion_candidates_recognition('That was good',1)
# x = dict_to_result(result)

In [ ]:
dff.tail()

,id,title,text,label,tokens,refined_tokens,index
62713,72127,WIKILEAKS EMAIL SHOWS CLINTON FOUNDATION FUNDS...,an email released by wikileaks on sunday appea...,1,[an email released by wikileaks on sunday appe...,[an email released by wikileaks on sunday appe...,62713
62714,72129,Russians steal research on Trump in hack of U....,washington (reuters) - hackers believed to be ...,0,[washington reuters hackers believed to be wor...,[washington reuters hackers believed to be wor...,62714
62715,72130,WATCH: Giuliani Demands That Democrats Apolog...,"you know, because in fantasyland republicans n...",1,[you know because in fantasyland republicans n...,[you know because in fantasyland republicans n...,62715
62716,72131,Migrants Refuse To Leave Train At Refugee Camp...,migrants refuse to leave train at refugee camp...,0,[migrants refuse to leave train at refugee cam...,[migrants refuse to leave train at refugee cam...,62716
62717,72132,Trump tussle gives unpopular Mexican leader mu...,mexico city (reuters) - donald trump’s combati...,0,[mexico city reuters donald trumps combative s...,[mexico city reuters donald trumps combative s...,62717


In [ ]:
dff['emotion_score_dict'] = dff['index'].parallel_apply(get_nearest_neighbours)

In [ ]:
dff['emotion_score_dict_tokenized'] = dff['index'].parallel_apply(get_nearest_neighbours_tokenized)

In [ ]:
print(dff['emotion_score_dict'][0])
print(dff['emotion_score_dict_tokenized'][0])

{'trust': 0.0, 'disgust': 0.002, 'anticipation': 0.002, 'sadness': 0.003, 'surprise': 0.003, 'joy': 0.008, 'fear': 0.009, 'anger': 0.014}
[{'trust': 0.0, 'sadness': 0.401, 'disgust': 1.198, 'anger': 1.78, 'anticipation': 2.367, 'surprise': 2.521, 'joy': 4.437, 'fear': 5.531}, {'trust': 0.0, 'anticipation': 0.021, 'joy': 0.026, 'surprise': 0.042, 'sadness': 0.102, 'disgust': 0.119, 'fear': 0.121, 'anger': 0.366}, {'trust': 0.0, 'sadness': 0.024, 'disgust': 0.026, 'surprise': 0.058, 'joy': 0.101, 'anticipation': 0.137, 'fear': 0.145, 'anger': 0.167}, {'trust': 0.0, 'anticipation': 0.022, 'joy': 0.024, 'surprise': 0.076, 'disgust': 0.08, 'fear': 0.119, 'anger': 0.405, 'sadness': 0.41}, {'anticipation': 0.0, 'trust': 0.0, 'joy': 0.0, 'fear': 0.042, 'surprise': 0.064, 'disgust': 0.266, 'sadness': 0.506, 'anger': 0.678}, {'trust': 0.0, 'disgust': 0.05, 'surprise': 0.052, 'anticipation': 0.071, 'fear': 0.075, 'joy': 0.096, 'sadness': 0.105, 'anger': 0.217}, {'trust': 0.0, 'disgust': 0.0, 'sad

In [ ]:
dff['highest_eight_label'] = dff['emotion_score_dict'].parallel_apply(dict_to_result)

In [ ]:
def most_frequent(List):
    return max(set(List), key = List.count)
dff['sequence_emotion'] = dff['emotion_score_dict_tokenized'].parallel_apply(dict_to_result_tokenized)
dff['highest_eight_label_tokenized'] = dff['sequence_emotion'].apply(most_frequent)

In [ ]:
from collections import Counter

def con_dicts(list_dicts):
  c = Counter()
  for d in list_dicts:
      c.update(d)
  final = dict(c)
  final = {k: round(v,3) for k, v in sorted(final.items(), key=lambda item: item[1])}
  return final

dff['cum_emotion_score_dict'] = dff['emotion_score_dict_tokenized'].apply(con_dicts)
print(dff['cum_emotion_score_dict'][0])

{'trust': 0.0, 'disgust': 3.49, 'surprise': 5.183, 'sadness': 5.862, 'fear': 11.354, 'joy': 11.476, 'anger': 11.801, 'anticipation': 12.214}


In [ ]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   id                             20 non-null     int64 
 1   title                          20 non-null     object
 2   text                           20 non-null     object
 3   label                          20 non-null     int64 
 4   tokens                         20 non-null     object
 5   refined_tokens                 20 non-null     object
 6   index                          20 non-null     int64 
 7   embedding                      20 non-null     object
 8   embedding_text                 20 non-null     object
 9   emotion_score_dict_tokenized   20 non-null     object
 10  emotion_score_dict             20 non-null     object
 11  highest_eight_label            20 non-null     object
 12  sequence_emotion               20 non-null     object
 13  highest

In [ ]:
dff.head()

,id,title,text,label,tokens,refined_tokens,index,embedding,embedding_text,emotion_score_dict_tokenized,emotion_score_dict,highest_eight_label,sequence_emotion,highest_eight_label_tokenized,cum_emotion_score_dict
0,66765,Republican wins Montana special election despi...,"bozeman, mont. (reuters) - republican greg gia...",0,"[bozeman mont, reuters republican greg gianfor...","[bozeman mont, reuters republican greg gianfor...",0,"[[-0.4079035818576813, -0.16996921598911285, 0...","[-0.5478277206420898, -0.6312752366065979, 0.1...","[{'trust': 0.0, 'sadness': 0.401, 'disgust': 1...","{'trust': 0.0, 'disgust': 0.002, 'anticipation...",anger,"[fear, anger, anger, sadness, anger, anger, an...",anger,"{'trust': 0.0, 'disgust': 3.49, 'surprise': 5...."
1,23592,China defends ally Pakistan after Trump criticism,beijing (reuters) - china defended its ally pa...,0,[beijing reuters china defended its ally pakis...,[beijing reuters china defended its ally pakis...,1,"[[-0.8079450726509094, 0.12918508052825928, -0...","[-0.8292466402053833, -0.11467402428388596, -0...","[{'trust': 0.0, 'disgust': 0.027, 'surprise': ...","{'trust': 0.0, 'disgust': 0.001, 'surprise': 0...",anger,"[anger, anticipation, joy, fear, anger, anger,...",anger,"{'trust': 0.0, 'disgust': 0.414, 'surprise': 0..."
2,48564,"Istanbul Airport, Benghazi, Pat Summitt: Your ...",(want to get this briefing by email? here’s th...,0,"[want to get this briefing by email, heres the...","[want to get this briefing by email, heres the...",2,"[[0.6069175601005554, -0.2507239580154419, 0.1...","[-0.22512289881706238, -0.4214745759963989, 0....","[{'trust': 0.0, 'surprise': 0.0, 'disgust': 0....","{'trust': 0.0, 'surprise': 0.0, 'disgust': 0.0...",fear,"[anticipation, anticipation, joy, anticipation...",anticipation,"{'trust': 0.0, 'disgust': 16.546, 'surprise': ..."
3,35067,"After Russia, Iran seeks deal for long-term Sy...","herzliya, israel (reuters) - the israeli intel...",0,[herzliya israel reuters the israeli intellige...,[herzliya israel reuters the israeli intellige...,3,"[[-0.4786793887615204, -0.25885164737701416, -...","[-0.7368257641792297, -0.11370693892240524, -0...","[{'trust': 0.0, 'disgust': 0.004, 'surprise': ...","{'trust': 0.0, 'disgust': 0.004, 'surprise': 0...",anger,"[anger, sadness, fear, joy, anticipation, anti...",anticipation,"{'trust': 0.0, 'surprise': 0.375, 'disgust': 0..."
4,65234,"Mugabe set to appoint woman deputy, all eyes o...",harare (reuters) - zimbabwean president robert...,0,[harare reuters zimbabwean president robert mu...,[harare reuters zimbabwean president robert mu...,4,"[[-0.40314391255378723, -0.5458259582519531, 0...","[-0.6361314058303833, -0.6486194133758545, -0....","[{'trust': 0.0, 'disgust': 0.01, 'sadness': 0....","{'trust': 0.0, 'disgust': 0.002, 'surprise': 0...",fear,"[fear, sadness, anger, anger, fear, fear, ange...",anger,"{'trust': 0.0, 'surprise': 1.564, 'disgust': 3..."


In [ ]:
out_dff = dff.copy()
out_dff.drop(['emotion_score_dict_tokenized','index'], axis=1, inplace=True)

In [ ]:
out_dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   id                             2000 non-null   int64 
 1   title                          1986 non-null   object
 2   text                           2000 non-null   object
 3   label                          2000 non-null   int64 
 4   tokens                         2000 non-null   object
 5   refined_tokens                 2000 non-null   object
 6   embedding                      2000 non-null   object
 7   embedding_text                 2000 non-null   object
 8   emotion_score_dict             2000 non-null   object
 9   highest_eight_label            1999 non-null   object
 10  sequence_emotion               2000 non-null   object
 11  highest_eight_label_tokenized  2000 non-null   object
 12  cum_emotion_score_dict         2000 non-null   object
dtypes: 

In [ ]:
out_dff.to_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kirishnni Emotion Extraction/Results/Explainability/ISOT_Explainability.csv",index=False)